In [25]:
# plots atrophy output from SIR simulator

import pandas as pd
import nibabel as nib
import nilearn.plotting
import numpy as np
import math

from nilearn import datasets
from nilearn import surface

lyso_score = pd.read_csv('./data/pcs/lyso_score.csv')
ubq_score = pd.read_csv('./data/pcs/ubq_score.csv')
risk_score = pd.read_csv('./data/pcs/risk_score.csv')

In [28]:
atlas_img = nib.load('./data/desikan_killiany_1.5mm.nii.gz')
# determine atlas shape
x, y, z = atlas_img.shape
atlas_data = atlas_img.get_fdata()


# setup
score = risk_score
# setup image header
score_header = atlas_img.header.copy()
score_header['datatype'] = 16
score_affine = atlas_img.affine.copy()

# setup surface for plot_img_on_surf
fsaverage = datasets.fetch_surf_fsaverage()
curv_left = surface.load_surf_data(fsaverage.curv_left)
curv_left_sign = np.sign(curv_left)

In [29]:
# pc = 0
for pc in range(40):
    score_data = np.zeros((x,y,z))
    for i in range (0, x):
        for j in range (0, y):
            for k in range (0, z):
                roi = int(atlas_data[i][j][k])
                vox_score = 0
                if roi > 0 and roi < 42:
                    vox_score = score.iloc[roi-1][pc]
                    if vox_score > 0:
                        vox_score = math.log2(vox_score)
                    if vox_score < 0:
                        vox_score = -math.log2(-vox_score)
                score_data[i][j][k] = vox_score
    score_img = nib.nifti1.Nifti1Image(score_data, score_affine, header=score_header) 

    texture = surface.vol_to_surf(score_img, fsaverage.pial_left)
    nilearn.plotting.plot_img_on_surf(score_img,
                                     views = ['lateral', 'medial'],
                                     hemispheres = ['left'],
                                     colorbar = True,
                                     inflate = True,
                                     output_file = "./data/pcs/risk_pc" + str(pc+1))
# nilearn.plotting.show()